<a href="https://colab.research.google.com/github/musman2012/ml-data_science/blob/master/ARM_Linear_Reg_Complete_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Analysis of Battery Data

In [0]:
import platform
import dill
dill.load_session('notebook_env.db')
platform.architecture()

('64bit', 'WindowsPE')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#!pip install tensorflow==2.0.0-alpha0 

In [4]:
'''
authored by Muhammad Usman (musman14@student.bradford.ac.uk)
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

FILE_PATH = "/content/drive/My Drive/Colab Notebooks/batteries_processed.csv"
bat_data = pd.read_csv(FILE_PATH, error_bad_lines = False)    # reading the data from csv file

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [5]:
data_columns = bat_data.columns
bat_data.shape

(5000000, 51)

In [22]:
input_data = bat_data.copy()
columns_to_drop = ['id', 'vin', 'vin_prefix', 'record_id','recorded_time', 'actual_time', 'arrival_time', 'powermode', 'mheader_vin', 'mheader_time', 'mheader_type', 'message','cat4', 'cat6', 'cat7', 'inlet_coolant_temp','cooling_energy_used', 'max_soh_cell', 'power_soh', 'min_soh', 'max_soh', 'soh','min_soh_cell_id','imbalance_percent', 'parkingdata_hvbattavgsocoatevent', 'parkingdata_hvbattavtempatevent']
input_data = input_data.drop(columns=columns_to_drop)
input_data.head()

## keep vin_prefix for input variables
## min_voltage is highly correlated with traction voltage and max_voltage
## 

,odometer,ambient_temp,vehicle_speed,balancing_trigger_cell,balancing_status,min_voltage,min_temp,traction_current,max_voltage,imbalance,traction_voltage,min_temp_cell,min_voltage_cell,max_voltage_cell,max_temp,max_temp_cell,fuse_temp,emodule_temp,thermal_manager_mode,outlet_coolant_temp,fast_charge_count,cycle,num_cycle,fast_charge,trip_distance
0,10619,6.5,0.0,22,noBalancing,4.107,12,0.675,4.153,0.046,445,15,22,96,14,1,870,135,activeHeating,195,0,storage,1,False,0
1,9531,4.0,0.0,22,noBalancing,3.983,6,7.375,4.008,0.025,432,16,36,21,7,1,870,90,idle,55,0,storage,1,False,-1088
2,9531,4.0,0.0,22,noBalancing,3.989,6,7.600,4.023,0.034,431,16,36,76,7,1,870,90,idle,55,0,storage,1,False,0
3,9531,4.0,0.0,22,noBalancing,3.984,6,6.850,4.020,0.036,431,16,36,79,7,1,870,90,idle,55,0,storage,1,False,0
4,9531,4.0,0.0,22,noBalancing,3.980,6,7.000,4.015,0.035,431,16,36,85,7,1,870,90,idle,55,0,storage,1,False,0


In [0]:
output_data = bat_data.soh

In [0]:
#!pip install sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics


In [0]:
print(input_data.info())

In [0]:
input_data.describe()

### Replacing String Categorical Variables to Numeric Categorical Features

In [24]:
object_attr = ['balancing_status', 'thermal_manager_mode', 'cycle', 'fast_charge']
for attr in object_attr:
    print(attr)
    print(bat_data[attr].value_counts())
    

balancing_status
noBalancing         3572630
passiveBalancing    1426264
initialValue           1106
Name: balancing_status, dtype: int64
thermal_manager_mode
idle                2647812
activeHeating       2340570
passingCooling         5702
thermalBalancing       4862
initialValue            558
activeCooling           496
Name: thermal_manager_mode, dtype: int64
cycle
charge     3595456
drive       937049
storage     467495
Name: cycle, dtype: int64
fast_charge
False    4979009
True       20991
Name: fast_charge, dtype: int64


In [0]:
balancing_kvp = {'noBalancing' : 1, 'passiveBalancing' : 2, 'initialValue' : 3}
thermal_kvp = {'idle' : 1, 'activeHeating' : 2, 'passingCooling' : 3, 'thermalBalancing' : 4, 'initialValue' : 5, 'activeCooling' : 6}
cycle_kvp = {'charge' : 1, 'drive' : 2, 'storage' : 3}
fast_charge_kvp = {False : 0, True : 1}

In [0]:
input_data[object_attr[0]] = input_data[object_attr[0]].replace(balancing_kvp)
input_data[object_attr[1]] = input_data[object_attr[1]].replace(thermal_kvp)
input_data[object_attr[2]] = input_data[object_attr[2]].replace(cycle_kvp)
input_data[object_attr[3]] = input_data[object_attr[3]].replace(fast_charge_kvp)
#fast_charge_kvp = {False : -1, True : 1}

In [27]:
#input_data = input_data.drop(columns=['recorded_time'])
input_data.head()

,odometer,ambient_temp,vehicle_speed,balancing_trigger_cell,balancing_status,min_voltage,min_temp,traction_current,max_voltage,imbalance,traction_voltage,min_temp_cell,min_voltage_cell,max_voltage_cell,max_temp,max_temp_cell,fuse_temp,emodule_temp,thermal_manager_mode,outlet_coolant_temp,fast_charge_count,cycle,num_cycle,fast_charge,trip_distance
0,10619,6.5,0.0,22,1,4.107,12,0.675,4.153,0.046,445,15,22,96,14,1,870,135,2,195,0,3,1,0,0
1,9531,4.0,0.0,22,1,3.983,6,7.375,4.008,0.025,432,16,36,21,7,1,870,90,1,55,0,3,1,0,-1088
2,9531,4.0,0.0,22,1,3.989,6,7.600,4.023,0.034,431,16,36,76,7,1,870,90,1,55,0,3,1,0,0
3,9531,4.0,0.0,22,1,3.984,6,6.850,4.020,0.036,431,16,36,79,7,1,870,90,1,55,0,3,1,0,0
4,9531,4.0,0.0,22,1,3.980,6,7.000,4.015,0.035,431,16,36,85,7,1,870,90,1,55,0,3,1,0,0


In [0]:
input_data.replace([np.inf, -np.inf], np.nan)

In [29]:
input_data.isnull().sum().sum()
#input_data['imbalance_percent'].value_counts()

0

## Test/Train Split of 20/80

In [0]:
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=0)

## Applying Decision Tree Regression

In [0]:
from sklearn.tree import DecisionTreeRegressor

reg_tree = DecisionTreeRegressor()

In [32]:
reg_tree.fit(X_train, y_train)
reg_tree_pred = reg_tree.predict(X_test)

'\nplt.figure(figsize=(10, 6))\nplt.plot(X_test, f(X_test), "b")\nplt.scatter(X_train, y_train, c="b", s=20)\nplt.plot(X_test, reg_tree_pred, "g", lw=2)\nplt.xlim([-5, 5])\nplt.title("Decision tree regressor, MSE = %.2f" % (np.sum((y_test - reg_tree_pred) ** 2) / n_test))\nplt.show()\n'

In [40]:
from sklearn.model_selection import cross_val_score

score = reg_tree.score(X_test, y_test)
print(score * 100)

95.30721088313385


In [0]:
#regressor = LinearRegression()  
#regressor.fit(X_train, y_train)

In [0]:
#sns.countplot(bat_data['powermode'])